In [1]:
import json
import pandas as pd
import csv
from pandas import read_excel
import ftputil
import time
from Bio import Entrez
import sys

In [2]:
def get_assembly_id(acc_id):
    """Get Assembly ID from Accession ID"""
    from Bio import Entrez
    handle = Entrez.esearch(db='assembly', term=acc_id)
    record = Entrez.read(handle)
    handle.close()
    assembly = record["IdList"][0]
    return assembly

def get_assembly_summary(acc_id):
    """Get esummary from an assembly ID"""
    from Bio import Entrez
    #provide your own mail here
    Entrez.email = #email
    assembly = get_assembly_id(acc_id)
    esummary_handle = Entrez.esummary(db="assembly", id=assembly, report="full")
    esummary_record = Entrez.read(esummary_handle)
    genbank_id = esummary_record['DocumentSummarySet']['DocumentSummary'][0]['Synonym']['Genbank']
    refseq_id = esummary_record['DocumentSummarySet']['DocumentSummary'][0]['Synonym']['RefSeq']
    return([assembly, genbank_id, refseq_id])

In [3]:
INPUT_TSV_FILENAME = "../../CF_isolates_BioSampleID.csv"
tsv_df = pd.read_csv(INPUT_TSV_FILENAME, sep='\t')
tsv_df.head()

,Genome Name / Sample Name,Unnamed: 0,Sales order,Item number,Name,Product,Level,BEI_accn,Organism,Strain_guess,...,Chromosomal Cassette Gene Count,Chromosomal Cassette Gene %,Chromosomal Cassette Count,Biosynthetic Cluster Gene Count * assembled,Biosynthetic Cluster Gene % * assembled,Biosynthetic Cluster Count * assembled,InterPro Count,InterPro %,Unnamed: 215,Match Status
0,Pseudomonas_aeruginosa_SCV20265_uid232358,NaN,NaN,NaN,Pseudomonas_aeruginosa_SCV20265_uid232358,NaN,NaN,NaN,Pseudomonas_aeruginosa_SCV20265_uid232358,SCV20265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pseudomonas_aeruginosa_M18_uid162089,NaN,NaN,NaN,Pseudomonas_aeruginosa_M18_uid162089,NaN,NaN,NaN,Pseudomonas_aeruginosa_M18_uid162089,M18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Enterobacter_aerogenes_EA1509E_uid187411,NaN,NaN,NaN,Enterobacter_aerogenes_EA1509E_uid187411,NaN,NaN,NaN,Enterobacter_aerogenes_EA1509E_uid187411,EA1509E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stenotrophomonas_maltophilia_K279a_uid61647,NaN,NaN,NaN,Stenotrophomonas_maltophilia_K279a_uid61647,NaN,NaN,NaN,Stenotrophomonas_maltophilia_K279a_uid61647,K279a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pseudomonas_aeruginosa_LESB58_uid59275,NaN,NaN,NaN,Pseudomonas_aeruginosa_LESB58_uid59275,NaN,NaN,NaN,Pseudomonas_aeruginosa_LESB58_uid59275,LESB58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
INPUT_JSON_FILENAME = "../../e_lenta_submission_template.json"

INSTRUMENT_METHOD_LABEL = "Seed Grant TOF"
EXTRACTION_METHOD = "SeedGrant_Extraction"
SAMPLE_PREP = "metagenomic_mouse_fecal"

template_json = json.loads(open(INPUT_JSON_FILENAME).read())

In [5]:
list_acc = tsv_df['Biosample Accession'].tolist()
genbank_dic = {}

for idx,acc_id in enumerate(list_acc):
    try:
        genbank_dic[acc_id] = (get_assembly_summary(acc_id))
    except:
        print("Accesion ID not found: " + acc_id)
        genbank_dic[acc_id] = (['-', '-', '-'])

    print(acc_id)
    print( str(idx+1) + "/" + str(len(list_acc)))

    time.sleep(4) 

SAMN02415141
1/5
SAMN02603849
2/5
SAMEA3138432
3/5
SAMEA1705934
4/5
SAMEA1705916
5/5


In [7]:
print(genbank_dic)
entrez_df = pd.DataFrame(genbank_dic.items(), columns=['Biosample Accession','esummary_record'])
entrez_df[['Assembly_ID', 'GenBank_ID', 'RefSeq_ID']]= pd.DataFrame(entrez_df.esummary_record.values.tolist(), index= entrez_df.index)
entrez_df = entrez_df[['Biosample Accession','Assembly_ID', 'GenBank_ID', 'RefSeq_ID']]
pairing_df = tsv_df.merge(entrez_df, on=['Biosample Accession'])
print(pairing_df.head())

{'SAMN02415141': ['87511', 'GCA_000510305.1', 'GCF_000510305.1'], 'SAMN02603849': ['376268', 'GCA_000226155.1', 'GCF_000226155.1'], 'SAMEA3138432': ['530308', 'GCA_000334515.1', 'GCF_000334515.1'], 'SAMEA1705934': ['91088', 'GCA_000072485.1', 'GCF_000072485.1'], 'SAMEA1705916': ['45568', 'GCA_000026645.1', 'GCF_000026645.1']}
                     Genome Name / Sample Name  Unnamed: 0  Sales order  \
0    Pseudomonas_aeruginosa_SCV20265_uid232358         NaN          NaN   
1         Pseudomonas_aeruginosa_M18_uid162089         NaN          NaN   
2     Enterobacter_aerogenes_EA1509E_uid187411         NaN          NaN   
3  Stenotrophomonas_maltophilia_K279a_uid61647         NaN          NaN   
4       Pseudomonas_aeruginosa_LESB58_uid59275         NaN          NaN   

   Item number                                         Name  Product  Level  \
0          NaN    Pseudomonas_aeruginosa_SCV20265_uid232358      NaN    NaN   
1          NaN         Pseudomonas_aeruginosa_M18_uid162089    

In [8]:
# Preparing the genome tables

all_genome_links = []
for pair in pairing_df.to_dict(orient="records"):
    genome_dict = {}
    genome_dict["genome_label"] = pair["Biosample Accession"]
    genome_dict["genome_ID"] = {
        "genome_type" : "metagenome",
        "GenBank_NCBI_accession" : pair["GenBank_ID"]
    }
    genome_dict["BioSample_accession"] = pair["Biosample Accession"]
    all_genome_links.append(genome_dict)

In [10]:
print(all_genome_links)

[{'genome_label': 'SAMN02415141', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_000510305.1'}, 'BioSample_accession': 'SAMN02415141'}, {'genome_label': 'SAMN02603849', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_000226155.1'}, 'BioSample_accession': 'SAMN02603849'}, {'genome_label': 'SAMEA3138432', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_000334515.1'}, 'BioSample_accession': 'SAMEA3138432'}, {'genome_label': 'SAMEA1705934', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_000072485.1'}, 'BioSample_accession': 'SAMEA1705934'}, {'genome_label': 'SAMEA1705916', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_000026645.1'}, 'BioSample_accession': 'SAMEA1705916'}]


In [11]:
# Preparing the join table

all_paired_links = []
for pair in pairing_df.to_dict(orient="records"):
    link_dict = {}
    link_dict["genome_label"] = pair["Biosample Accession"]
    link_dict["metabolomics_file"] = pair["MS data accession, Sputum/ASM Media (data in MSV000080251)"]
    link_dict["sample_preparation_label"] = SAMPLE_PREP
    link_dict["extraction_method_label"] = EXTRACTION_METHOD
    link_dict["instrumentation_method_label"] = INSTRUMENT_METHOD_LABEL
    
    all_paired_links.append(link_dict)

In [13]:
print(all_paired_links)

[{'genome_label': 'SAMN02415141', 'metabolomics_file': 'ftp://massive.ucsd.edu/MSV000080251/peak/CFStrains/Strains/VVP011_TH_GA11_01_27002.mzXML', 'sample_preparation_label': 'metagenomic_mouse_fecal', 'extraction_method_label': 'SeedGrant_Extraction', 'instrumentation_method_label': 'Seed Grant TOF'}, {'genome_label': 'SAMN02603849', 'metabolomics_file': 'ftp://massive.ucsd.edu/MSV000080251/peak/CFStrains/Strains/VVP012_TH_GA12_01_27003.mzXML', 'sample_preparation_label': 'metagenomic_mouse_fecal', 'extraction_method_label': 'SeedGrant_Extraction', 'instrumentation_method_label': 'Seed Grant TOF'}, {'genome_label': 'SAMEA3138432', 'metabolomics_file': 'ftp://massive.ucsd.edu/MSV000080251/peak/CFStrains/Strains/VVP016_TH_GB4_01_27009.mzXML', 'sample_preparation_label': 'metagenomic_mouse_fecal', 'extraction_method_label': 'SeedGrant_Extraction', 'instrumentation_method_label': 'Seed Grant TOF'}, {'genome_label': 'SAMEA1705934', 'metabolomics_file': 'ftp://massive.ucsd.edu/MSV000080251/

In [14]:
# Merging it all together

template_json["genomes"] = all_genome_links
template_json["genome_metabolome_links"] = all_paired_links
with open("../../CF_isolates_json.json", "w") as output_file:
    output_file.write(json.dumps(template_json))

In [16]:
print(template_json)

{'version': '1', 'personal': {'submitter_name': 'Alexander Aksenov', 'submitter_orcid': 'https://orcid.org/0000-0002-9445-2248', 'PI_name': 'aaaksenov@ucsd.edu', 'PI_institution': 'University of California, San Diego', 'PI_email': 'pdorrestein@health.ucsd.edu'}, 'metabolomics': {'project': {'GNPSMassIVE_ID': 'MSV000083734', 'MaSSIVE_URL': 'https://gnps.ucsd.edu/ProteoSAFe/result.jsp?task=d80e741723a143348e0f196065427e57&view=advanced_view', 'molecular_network': '85c9922a8b8548e3a537dda24301673f'}}, 'experimental': {'sample_preparation': [{'medium_details': {'medium_type': 'liquid', 'medium': 'other', 'Other_medium': 'Brain Heart Infusion (BHI) Broth', 'Other_medium_link': 'https://microbeonline.com/brain-heart-infusion-bhi-broth-composition-preparation-and-uses/'}, 'growth_parameters': {}, 'aeration': {}, 'sample_preparation_method': 'E Lenta'}], 'extraction_methods': [{'extracted_material': 'cells_supernatant', 'resins': 'Oasis HLB', 'extraction_method': 'Oasis HLB', 'solvents': [{'ra